In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torchvision import datasets
#Neural Networks
import torch.nn as nn
import torch.nn.functional as F
#Visualize img
from torchvision.utils import make_grid
#Optimizer
import torch.optim as optim
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#mounting drive
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/Deep_Learning/MSFT.csv'
#imports for neural network 
from sklearn.preprocessing import MinMaxScaler

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
MSFT = pd.read_csv(data_dir)
# MSFT = MSFT.iloc[7000:,:]
MSFT;
display(MSFT)


,Date,Open,High,Low,Close,Adj Close,Volume
0,1986-03-13,0.088542,0.101562,0.088542,0.097222,0.062378,1031788800
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.064606,308160000
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.065720,133171200
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.064049,67766400
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.062935,47894400
...,...,...,...,...,...,...,...
8579,2020-03-26,148.399994,156.660004,148.369995,156.110001,156.110001,64568100
8580,2020-03-27,151.750000,154.889999,149.199997,149.699997,149.699997,57042300
8581,2020-03-30,152.440002,160.600006,150.009995,160.229996,160.229996,63420300
8582,2020-03-31,159.399994,164.779999,156.559998,157.710007,157.710007,77927200


In [7]:
MSFT.iloc[:,5]

0         0.062378
1         0.064606
2         0.065720
3         0.064049
4         0.062935
           ...    
8579    156.110001
8580    149.699997
8581    160.229996
8582    157.710007
8583    152.110001
Name: Adj Close, Length: 8584, dtype: float64

In [42]:
class Basenet(nn.Module):
  def __init__(self):
    self.train_set = None
    self.test_set = None
    self.trainloader = None
    self.testloader = None


  def convert_to_tensor(self,arr):
    return torch.from_numpy(arr).type(torch.Tensor)

  def split_data(self,stock, lookback,splitsize):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    #calc test size
    data = np.array(data)
    test_size = int(np.round(len(data)*splitsize))
    train_size = len(data) - test_size
    # test_set_size = int(np.round(0.2*data.shape[0]))
    # train_set_size = data.shape[0] - (test_set_size)
    self.train_set = data[:train_size,:]
    self.test_set = data[train_size:,:]
    x_train = self.c (data[:train_size,:-1])
    y_train = data[:train_size,-1]
    print("Training Size :",len(x_train))
    x_test = data[train_size:,:-1]
    y_test = data[train_size:,-1]

    print("Testing Size :",len(x_test))
    
    return True

  def loaders(self,batch_size):
    self.trainloader = torch.utils.data.DataLoader(self.train_set, batch_size=batch_size,
                                                      shuffle=True, num_workers=2)
    print("Training Size :",len(self.train_set))

    self.testloader = torch.utils.data.DataLoader(self.test_set, batch_size=batch_size,
                                                    shuffle=False, num_workers=2)
    print("Testing Size :",len(self.test_set))
  

In [43]:
b = Basenet()
b.split_data(MSFT["Adj Close"], 20,.2)
b.loaders(2)

Training Size : 6851
Testing Size : 1713
Training Size : 6851
Testing Size : 1713


# Dataset : Microsoft Stock

In [4]:
# MSFT.columns

#Display the graph of adjusted Closing prices for Microsoft stock 
# # plt.figure(figsize=(5, 4))
# plt.title("Plotting adj closing price vs time period")
# plt.plot(MSFT["Date"].values, MSFT["Adj Close"].values, label = "Microsoft Stock")
# # plt.xticks(MSFT["Date"].values, MSFT.iloc[::365,0], rotation ='vertical')
# plt.legend()
# plt.show()

#Graph

import plotly.graph_objects as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=MSFT['Date'],y=MSFT['Adj Close'],name='Closing Price'),secondary_y=False)
# fig.add_trace(go.Bar(x=MSFT['Date'],y=MSFT['Volume'],name='Volume'),secondary_y=True)
fig.show()


In [ ]:

price = data[['Close']]
scaler = MinMaxScaler(feature_range=(-1, 1))
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))

0       1986-03-13
365     1987-08-21
730     1989-01-31
1095    1990-07-12
1460    1991-12-19
1825    1993-06-01
2190    1994-11-08
2555    1996-04-19
2920    1997-09-29
3285    1999-03-12
3650    2000-08-21
4015    2002-02-07
4380    2003-07-22
4745    2004-12-31
5110    2006-06-14
5475    2007-11-26
5840    2009-05-08
6205    2010-10-19
6570    2012-03-30
6935    2013-09-13
7300    2015-02-26
7665    2016-08-08
8030    2018-01-19
8395    2019-07-03
Name: Date, dtype: object

ModuleNotFoundError: ignored